In [2]:
# !ls ~/data/ml-latest-small

README.txt  links.csv  models  movies.csv  ratings.csv	tags.csv  tmp


In [3]:
import sys; sys.path.append('/home/nikhil/fastai/')
import fastai

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch

from fastai.learner import *
from fastai.column_data import *

In [19]:
PATH = '/home/nikhil/data/ml-latest-small/'

1. forward layer in NN is matrix multiplication
1. backward is gradient

## create first NN with Pytorch

In [16]:
class DotProduct(nn.Module):
    def forward(self, a,b):
        return (a*b).sum(1)
    


### lets create array/ Tensor

In [8]:
a = T([
    [1.,2],
    [3.,4]
])
b = T([
    [2.,3],
    [5.,6]
])
a, b

(
  1  2
  3  4
 [torch.cuda.FloatTensor of size 2x2 (GPU 0)], 
  2  3
  5  6
 [torch.cuda.FloatTensor of size 2x2 (GPU 0)])

In [9]:
a*b


  2   6
 15  24
[torch.cuda.FloatTensor of size 2x2 (GPU 0)]

In [14]:
(a*b).sum(1)


  8
 39
[torch.cuda.FloatTensor of size 2 (GPU 0)]

In [17]:
model = DotProduct()
model(a,b)


  8
 39
[torch.cuda.FloatTensor of size 2 (GPU 0)]

### read csv files

In [87]:
ratings = pd.read_csv(PATH + 'ratings.csv')
movies = pd.read_csv(PATH + 'movies.csv')

In [89]:
u_uniq = ratings.userId.unique()
user2idx = {o:i for i,o in enumerate(u_uniq)}
ratings.userId = ratings.userId.apply(lambda x: user2idx[x])

m_uniq = ratings.movieId.unique()
movie2idx = {o:i for i,o in enumerate(m_uniq)}
ratings.movieId = ratings.movieId.apply(lambda x: movie2idx[x])
n_users=int(ratings.userId.nunique())
n_movies=int(ratings.movieId.nunique())

In [90]:
ratings.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [98]:
class EmbeddingDot(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=50):
        super().__init__()
        self.u = nn.Embedding(n_users, n_factors)
        self.m = nn.Embedding(n_movies, n_factors)
        self.u.weight.data.uniform_(0, 0.05) # initialize the value
        self.m.weight.data.uniform_(0, 0.05) # _ inplace calculation
        
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1]
        u, m = self.u(users), self.m(movies)
        return (u*m).sum(1).view(-1,1)

In [92]:
x = ratings.drop(['rating', 'timestamp'],axis=1)
y = ratings['rating'].astype(np.float32)

In [93]:
val_idxs = get_cv_idxs(len(ratings))

In [99]:
data = ColumnarModelData.from_data_frame(
        PATH, val_idxs, x, y, ['userId', 'movieId'], 64)


In [100]:
wd = 1e-5
model = EmbeddingDot(n_users, n_movies).cuda()

In [102]:
opt = optim.SGD(model.parameters(), 1e-1, weight_decay=wd, momentum=0.9)
fit(model, data, 3, opt, F.mse_loss)

epoch      trn_loss   val_loss                                 
    0      1.648509   1.650596  
    1      1.153318   1.306828                                 
    2      0.9425     1.226375                                  



[array([1.22637])]

In [103]:
set_lrs(opt, 0.01)

In [104]:
fit(model, data, 3, opt, F.mse_loss)

epoch      trn_loss   val_loss                                  
    0      0.693282   1.14871   
    1      0.698403   1.136662                                  
    2      0.660663   1.130741                                  



[array([1.13074])]

### Ratings range is 

In [106]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()
min_rating,max_rating

(0.5, 5.0)

In [113]:
n_factors = 50
[get_emb(*o) for o in [
            (n_users, n_factors), (n_movies, n_factors), (n_users,1), (n_movies,1)
        ]]
        

[Embedding(671, 50),
 Embedding(9066, 50),
 Embedding(671, 1),
 Embedding(9066, 1)]

In [117]:
class EmbeddingDotBias(nn.Module):
    def __init__(self, n_users, n_movies):
        super().__init__()
        (self.u, self.m, self.ub, self.mb) = [get_emb(*o) for o in [
                (n_users, n_factors), (n_movies, n_factors), (n_users,1), (n_movies,1)
                ]]
        
    def forward(self, cats, conts):
        users, movies = cats[:,0], cats[:,1]
        um = self.u(users) * self.m(movies)
        # squeeze is going to be broadcasting, this will replicate a vector
        res = um.sum(1) + self.ub(users).squeeze() + self.mb(movies).squeeze()
        res = F.sigmoid(res) * (max_rating - min_rating) + min_rating
        return res

In [123]:
wd = 2e-4
model = EmbeddingDotBias(n_users, n_movies).cuda()

In [124]:
opt = optim.SGD(model.parameters(), 1e-1, weight_decay=wd, momentum=.9)


In [125]:
fit(model, data, 3, opt, F.mse_loss)

epoch      trn_loss   val_loss                                  
    0      0.849259   0.836409  
    1      0.838548   0.814627                                  
    2      0.771185   0.811317                                  



[array([0.81132])]